In [ ]:
user_matrix = pd.DataFrame()
user_matrix['userid'] = sorted(point['userid'].unique())
user_matrix = user_matrix.set_index(user_matrix['userid'])
user_matrix['busy'] = 1
user_matrix.head()

In [ ]:
land_matrix = pd.DataFrame()
land_matrix['landid'] = sorted(point['index_right'].unique())
land_matrix = land_matrix.set_index(land_matrix['landid'])

land_matrix['busy'] = 1 

len_user = len(user_matrix)
len_land = len(land_matrix)
d = 0.85 

In [ ]:
def user2land(land_matrix, user_matrix):    
    land_change = land_matrix.copy()
    land_change.iloc[:,1] = 0
    for i in range(len(point)):
        this_userid = point.iloc[i]['userid']
        this_landid = point.iloc[i]['index_right']
        land_change.loc[this_landid,'busy'] += (user_matrix.loc[this_userid,'busy'] \
        / len(point[(point['userid']==this_userid)]))
    return land_change

        
def land2user(land_matrix, user_matrix):
    user_change = user_matrix.copy()
    user_change.iloc[:,1] = 0
    for i in range(len(point)):
        this_userid = point.iloc[i]['userid']
        this_landid = point.iloc[i]['index_right']
        user_change.loc[this_userid,'busy'] += (land_matrix.loc[this_landid,'busy'] \
        / len(point[(point['index_right']==this_landid)]))
    return user_change

In [ ]:
land_change = user2land(land_matrix, user_matrix)
land_matrix.iloc[:,1] = (land_change.iloc[:,1]*d + (1-d)/len_land)   

user_change = land2user(land_matrix, user_matrix)
user_matrix.iloc[:,1] = (user_change.iloc[:,1]*d + (1-d)/len_user)  

In [ ]:
user_process = pd.DataFrame(index=user_matrix.index)
land_process = pd.DataFrame(index=land_matrix.index)

user_process[1] = user_matrix['busy']
land_process[1] = land_matrix['busy']

quit = False
vote_num = 1  
sigma = 0.001  
while not quit:
    vote_num += 1
    print(vote_num)
    
    land_change = user2land(land_matrix, user_matrix)
    land_change.iloc[:,1] = land_change.iloc[:,1]*d + (1-d)/len_land
    land_matrix = land_change.copy()
    user_change = land2user(land_matrix, user_matrix)
    user_change.iloc[:,1] = user_change.iloc[:,1]*d + (1-d)/len_user
    user_matrix = user_change.copy()
    
    user_process[vote_num] = user_change['busy'] 
    land_process[vote_num] = land_change['busy'] 
    
    if vote_num >2:
        this_user_change = user_change.iloc[:,1].values
        last_user_change = last_user.iloc[:,1].values
        this_land_change = land_change.iloc[:,1].values
        last_land_change = last_land.iloc[:,1].values
        
        a = np.linalg.norm(last_user_change-this_user_change)
        b = np.linalg.norm(last_land_change-this_land_change)
        
        print(a)
        print(b)
        
        if (a<sigma) and (b<sigma):
            quit = True    
    
    last_user = user_change.copy()
    last_land = land_change.copy()

In [ ]:
user_process.to_csv('user_process_busy.csv')
land_process.to_csv('land_process_busy.csv')